# Coronavirus (COVID 19)

## Overview

**Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.**

**Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.**

**The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.** 

**The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).**

**At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available.**

## Prevention

**To prevent infection and to slow transmission of COVID-19, do the following:**

**1) Wash your hands regularly with soap and water, or clean them with alcohol-based hand rub.**

**2) Maintain at least 1 metre distance between you and people coughing or sneezing.**

**3) Avoid touching your face.**

**4) Cover your mouth and nose when coughing or sneezing.**

**5) Stay home if you feel unwell.**

**6) Refrain from smoking and other activities that weaken the lungs.**

**7) Practice physical distancing by avoiding unnecessary travel and staying away from large groups of people.**


## Symptoms

**The COVID-19 virus affects different people in different ways.  COVID-19 is a respiratory disease and most infected people will develop mild to moderate symptoms and recover without requiring special treatment.  People who have underlying medical conditions and those over 60 years old have a higher risk of developing severe disease and death.**

**Common symptoms include:**

**1) fever**

**2) tiredness**

**3) dry cough.**

**Other symptoms include:**

**1) shortness of breath**

**2) aches and pains**

**3) sore throat**

**and very few people will report diarrhoea, nausea or a runny nose.**

##### In this notebook , we'll be looking at how india has been affected with this ongoing coronavirus (COVID 19) 

##### With some basic analysis and interesting visualizations

##### I will be continuosly updating this notebook with added features and visualizations.

**Download libraries**

In [ ]:
! pip install calmap

### IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import copy

#Data Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import folium
import calmap
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"

#Some styling
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

#displaying markdown
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

#Web scraping tools
#REQUESTS --> to fetch data from website
import requests
import json

#BEAUTIFULSOUP  -->parse HTML content 
from bs4 import BeautifulSoup  

#Showing full path of datasets
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# Disable warnings 
import warnings
warnings.filterwarnings('ignore')


#### Loading Datasets

In [ ]:
#India states co-ordinates
india_coord=pd.read_csv("/kaggle/input/indian-coordinates/Indian Coordinates.csv")

#India states population
population=pd.read_csv("/kaggle/input/covid19-in-india/population_india_census2011.csv")

#hospital
hospital_beds_india=pd.read_csv("/kaggle/input/covid19-in-india/HospitalBedsIndia.csv")
hospital_beds_gro = pd.read_csv("/kaggle/input/hospital-beds-grouped/hospital_beds_gro.csv")

#Age groups
age_group=pd.read_csv("/kaggle/input/covid19-in-india/AgeGroupDetails.csv")

#ICMR testing labs
ICMRtestinglabs = pd.read_csv("/kaggle/input/covid19-in-india/ICMRTestingLabs.csv")
ICMRstatetesting = pd.read_csv("/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv")

## India :- Statewise cases

In [ ]:
'''
Statewise Total Of cases
'''
import requests

url='https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise'
json_data=requests.get(url).json()

'''
Normalizing the dataset
'''
dataset = pd.io.json.json_normalize(json_data,record_path=[['data','statewise']])

#No need of creating separate dataset
dataset=dataset.rename(columns={'state':'State',
                               'confirmed':'Confirmed','recovered':'Recovered',
                               'deaths':'Deaths','active':'Active'})
#making a copy
import copy
india_total=dataset.copy()

'''
Data cleaning
'''
india_total['State']=india_total["State"].replace("Chhattisgarh",'Chattisgarh')
india_total['State']=india_total["State"].replace('Puducherry','Pondicherry')
india_total['State']=india_total["State"].replace('Telangana','Telengana')

india_total['Death Rate(per 100)']=round((india_total['Deaths']/india_total['Confirmed'])*100,2)
india_total['Recovery Rate(per 100)']=round((india_total['Recovered']/india_total['Confirmed'])*100,2)

#Final
india_total.style.background_gradient(cmap='YlOrBr',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Blues',subset=["Active"])\
                        .background_gradient(cmap='Oranges',subset=['Death Rate(per 100)'])\
                        .background_gradient(cmap='Greys',subset=['Recovery Rate(per 100)'])

### Covid19 Cases history

In [ ]:
'''
Covid19 Cases history
'''
import requests

url='https://api.rootnet.in/covid19-in/stats/history'
json_data=requests.get(url).json()

'''
Normalizing the dataset
'''
dataset = pd.io.json.json_normalize(json_data,record_path=[['data']])
dataset

'''
Extracting statewise data from test
'''

out_lst=[]
for i in range(0,len(dataset)):
    date = dataset.iloc[i]['day']
    ls = dataset.iloc[i]['regional']
    for ele in ls: #list of thw row
        ele.update({'date':date})
        inn_list=[]
        inn_list.append(ele.values())
        for i in ele: #i is each element in that row
            if i=='date':
                inn_list.insert(0,ele[i])
            elif i=='loc':
                inn_list.insert(1,ele[i])
            elif i=='confirmedCasesIndian':
                inn_list.insert(2,ele[i])
            elif i=='confirmedCasesForeign':
                inn_list.insert(3,ele[i])
            elif i=='discharged':
                inn_list.insert(4,ele[i])
            elif i=='deaths':
                inn_list.insert(5,ele[i])
            elif i=='totalConfirmed':
                inn_list.insert(6,ele[i])
        del inn_list[1]
        out_lst.append(inn_list)
        
'''
Creating new dataframe
'''
df=pd.DataFrame(out_lst,columns=['Date','State','Indian Cases','Foreign Cases','Recovered','Deaths','Confirmed'])

'''
Changing date to datetime format
'''
df['Date']=df['Date'].apply(pd.to_datetime,dayfirst=True)

#Final
#Making a copy
india=df.copy()

#Adding Active cases column
india['Active']=india["Confirmed"]-india['Recovered']-india['Deaths']

india['Death Rate(per 100)']=round(100*india['Deaths']/india['Confirmed'],2)
india['Recovery Rate(per 100)']=round(100*india['Recovered']/india['Confirmed'],2)
india['Death Rate(per 100)']=india['Death Rate(per 100)'].fillna(0)
india['Recovery Rate(per 100)']=india['Recovery Rate(per 100)'].fillna(0)

#there are two similar states Chattisgarh and Chhattisgarh, we'll rename it
india['State']=india["State"].replace("Chhattisgarh",'Chattisgarh')
india['State']=india["State"].replace('Puducherry','Pondicherry')

india.head()


### ICMR Testing dataset

In [ ]:
'''
Stats testing history
'''
import requests

url='https://api.rootnet.in/covid19-in/stats/testing/history'
json_data=requests.get(url).json()
#Normalize
test_data = pd.io.json.json_normalize(json_data['data'])

test_data.drop(columns=['source','totalIndividualsTested'],inplace=True)
#We had to drop the totalIndividualsTested feature as there are many nan values
test_data=test_data.rename(columns={"day":"Date"})
test_data['Date']=test_data['Date'].apply(pd.to_datetime , dayfirst = True)

test_data=test_data.fillna(0)

display(test_data)

### Data Cleaning

In [ ]:
#india_coord
india_coord=india_coord.rename(columns={"Name of State / UT":"State",
                                       "Latitude":"Lat","Longitude":"Long"})
#Population dataset
population=population.drop(columns=["Sno"])
population.rename(columns = {"State / Union Territory": "State"}, inplace = True)
population['Area']=population['Area'].str.replace(',','').str.split("km").str[0].astype(int)
population['Density']=population['Density'].str.replace(',','').str.split("/").str[0].astype(float)
population['State']=population['State'].replace({'Chhattisgarh':'Chattisgarh',
                                                 'Puducherry':'Pondicherry'})



#### Merging all the datasets

In [ ]:
#Merge both the datasets india and india_coord

india=pd.merge(india,india_coord,on=['State'],how='inner')
india=pd.merge(india,population,on=['State'],how='inner')

In [ ]:
'''
Infection Rate(per million) per million
'''
india['Infection Rate(per million)'] = india['Confirmed']/india['Population']
india['Infection Rate(per million)'] = round(india['Infection Rate(per million)']*1000000,2)

In [ ]:
'''
Data Cleaning
'''
#Drop
hospital_beds_india=hospital_beds_india[['State/UT','NumPublicBeds_HMIS',
                                        'NumUrbanBeds_NHP18']]

#Change column names
hospital_beds_india=hospital_beds_india.rename(columns={"State/UT":"State"})

#Changing names
hospital_beds_india['State']=hospital_beds_india['State'].replace('Andaman & Nicobar Islands','Andaman and Nicobar Islands')
hospital_beds_india['State']=hospital_beds_india['State'].replace('Chhattisgarh','Chattisgarh')
hospital_beds_india['State']=hospital_beds_india['State'].replace('Jammu & Kashmir','Jammu and Kashmir')
hospital_beds_india['State']=hospital_beds_india['State'].replace('Puducherry','Pondicherry')
hospital_beds_india['State']=hospital_beds_india['State'].replace('Telangana','Telengana')

#Missing values
#hospital_beds_india['NumSubDistrictHospitals_HMIS']=hospital_beds_india['NumSubDistrictHospitals_HMIS'].fillna(0)

#By hospital_beds.info() ,  we see that there are few objects
#The problem arises in pd.to_numeric for All India row

All_India=hospital_beds_india[hospital_beds_india['State']=='All India']
hospital_beds_india=hospital_beds_india[hospital_beds_india['State']!='All India']

'''
Changing objects to numeric
'''
for col in hospital_beds_india.columns:
    if col!='State':
        if hospital_beds_india[col].dtype=='O':
            hospital_beds_india[col]=pd.to_numeric(hospital_beds_india[col])

#Drop NAN
hospital_beds_india=hospital_beds_india.dropna()

#Reset index
hospital_beds_india=hospital_beds_india.reset_index(drop=True)

#Converting integer values back to float

for col in hospital_beds_india.columns:
    if col!='State':
        if hospital_beds_india[col].dtype=='int64':
            hospital_beds_india[col]=hospital_beds_india[col].astype(float)

#hospital grouped data
hospital_beds_gro.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
'''
Merging how='left'
''' 
hospital_beds_india = pd.merge(hospital_beds_india,hospital_beds_gro,on=['State'],how='left')
hospital_beds_india = hospital_beds_india.fillna(0)

In [ ]:
#Now our hospital dataset is ready
#We are ready to merge

india=pd.merge(india,hospital_beds_india,on=['State'],how='inner')

### Covid 19 Dashboard

**Total Confirmed Cases**

In [ ]:
india_total['Confirmed'].sum()

**Total Deaths**

In [ ]:
india_total['Deaths'].sum()

**Total Recovered Cases**

In [ ]:
india_total['Recovered'].sum()

**Total Active Cases**

In [ ]:
india_total['Active'].sum()

### Trend of cases in india

In [ ]:
india_latest=india[india['Date'] == max(india['Date'])]

#Creating Empty Map
m=folium.Map(location=[20.5937, 78.9629], zoom_start=14,max_zoom=4,min_zoom=3,tiles="Stamen Toner",
            height = 600,width = '70%')

#Fill the latest data in the map

for i in range(0,len(india_latest)):
    folium.Circle(location=[india_latest.iloc[i]["Lat"],india_latest.iloc[i]["Long"]],
                 color="crimson",
                  radius=int(india_latest.iloc[i]["Confirmed"]*50),
                  tooltip='<li><bold>State : '+str(india_latest.iloc[i]["State"])+
                          '<li><bold>Confirmed : '+str(india_latest.iloc[i]["Confirmed"])+
                          '<li><bold>Deaths : '+str(india_latest.iloc[i]["Deaths"]),
                  fill=True
                 ).add_to(m)
    
m

In [ ]:
'''
Spread of coronavirus over time
'''
india_grouped = india.groupby(['Date'])['Confirmed','Deaths','Recovered',
                                        'Active'].sum().reset_index()
india_grouped['Date']=india_grouped['Date'].dt.strftime("%m/%d/%Y")

fig = go.Figure()
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Confirmed'],
                    mode='lines+markers',marker_color='purple',name='Confirmed'))
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Recovered'],
                    mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Deaths'],
                    mode='lines+markers',marker_color='red',name='Deaths'))
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Active'],
                    mode='lines+markers',marker_color='blue',name='Active'))
fig.update_layout(title_text='Trend of cases in india over time')
fig.show()

#### The important part to notice here is that the number of recovered cases are increasing day by day which is a good sign.


**Death Rate(per 100),Recovery Rate(per 100),Infection Rate(per million)**

In [ ]:
india_grouped = india.groupby(['Date'])[['Death Rate(per 100)','Recovery Rate(per 100)',
                                         'Infection Rate(per million)']].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Death Rate(per 100)'],
                    mode='lines+markers',marker_color='red',name='Death Rate(per 100)'))
fig.update_layout(title_text='Death rate in india over time',height=400,width=800)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Recovery Rate(per 100)'],
                    mode='lines+markers',marker_color='green',name='Recovery Rate(per 100)'))
fig.update_layout(title_text='Recovery rate in india over time',height=400,width=800)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Infection Rate(per million)'],
                    mode='lines+markers',marker_color='purple',name='Infection Rate(per million)'))
fig.update_layout(title_text='Infection rate in india over time',height=400,width=800)
fig.show()

**Infection rate vs Death rate vs Recovery rate**

In [ ]:
india_grouped = india.groupby(['Date'])['Confirmed','Deaths','Recovered',
                                        'Active','Infection Rate(per million)',
                                       'Recovery Rate(per 100)','Death Rate(per 100)'].sum().reset_index()
india_grouped['Date']=india_grouped['Date'].dt.strftime("%m/%d/%Y")

fig = go.Figure()
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Recovery Rate(per 100)'],
                    mode='lines+markers',marker_color='green',name='Recovery Rate(per 100)'))
fig.add_trace(go.Scatter(x=india_grouped['Date'], y=india_grouped['Death Rate(per 100)'],
                    mode='lines+markers',marker_color='orange',name='Death Rate(per 100)'))

fig.update_layout(title_text='Rate of cases in india over time')
fig.show()

**Recovery rate and Death rate of each state**

In [ ]:
india_grouped=india.groupby('State')[['Recovery Rate(per 100)','Death Rate(per 100)']].max().reset_index()
#Recovery Rate(per 100)
fig = px.bar(india_grouped.sort_values(by='Recovery Rate(per 100)'),x='Recovery Rate(per 100)',
            y='State',color='Recovery Rate(per 100)',orientation='h',height=800,text='Recovery Rate(per 100)',
            title = 'Recovery Rate(per 100)')
fig.update_traces(textposition='outside')
fig.show()
#Death Rate(per 100)
fig = px.bar(india_grouped.sort_values(by='Death Rate(per 100)'),x='Death Rate(per 100)',
            y='State',color='Death Rate(per 100)',orientation='h',height=800,text='Death Rate(per 100)',
            title = 'Death Rate(per 100)')
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
india_grouped=india.groupby(['State'])[['Confirmed','Recovered','Deaths','Active',
                                        'Infection Rate(per million)','Death Rate(per 100)',
                                       'Recovery Rate(per 100)']].max().reset_index()


In [ ]:
import IPython
IPython.display.HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

In [ ]:
import plotly.express as px
#Confirmed
fig=px.bar(india_grouped.sort_values(by='Confirmed'),x='Confirmed',y='State',
          text='Confirmed',orientation='h',
          height=800,title='Confirmed cases',color='Confirmed')
fig.update_traces(textposition='outside')
fig.show()
#Deaths
fig=px.bar(india_grouped.sort_values(by='Deaths'),x='Deaths',y='State',
          color = 'Deaths',text='Deaths',orientation='h',
          height=800,title='Deaths')
fig.update_traces(textposition='outside')
fig.show()
#Recovered cases
fig=px.bar(india_grouped.sort_values(by='Recovered'),x='Recovered',y='State',
          color = 'Recovered',text='Recovered',orientation='h',
          height=800,title='Recovered cases')
fig.update_traces(textposition='outside')
fig.show()
#Most Active cases
fig=px.bar(india_grouped.sort_values(by='Active'),x='Active',y='State',
          color='Active',text='Active',orientation='h',
          height=800,title='Active cases')
fig.update_traces(textposition='outside')
fig.show()

#### Active vs Recovered vs Deaths

In [ ]:
temp=india_grouped.sort_values(by='Confirmed')
fig = go.Figure(data=[
go.Bar(name='Active', y=temp['State'], x=temp['Active'], 
       orientation='h',marker_color='blue'),
go.Bar(name='Recovered', y=temp['State'], x=temp['Recovered'], 
       orientation='h',marker_color='green'),
go.Bar(name='Deaths', y=temp['State'], x=temp['Deaths'], 
       orientation='h',marker_color='red')])
fig.update_layout(barmode='stack',width=1000, height=800)

fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.update_layout(title_text='Active Cases,Recovered,Deaths in Different States of India',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()

#### Line Charts

In [ ]:
date_grouped = india.groupby(['Date','State'])['Confirmed','Deaths','Recovered','Active','Infection Rate(per million)'].sum().reset_index()
date_grouped['Date']=date_grouped['Date'].dt.strftime("%m/%d/%Y")

fig = px.line(date_grouped,x='Date',y='Confirmed',title='Confirmed cases over time',color='State')
fig.show()
fig = px.line(date_grouped,x='Date',y='Deaths',title='Fatality cases over time',color='State')
fig.show()
fig = px.line(date_grouped,x='Date',y='Recovered',title='Recovered cases over time',color='State')
fig.show()
fig = px.line(date_grouped,x='Date',y='Infection Rate(per million)',title='Infection rate over time',color='State')
fig.show()

### Daily Cases

In [ ]:
india_daily=india[['Date','State','Confirmed','Recovered','Deaths','Active']]
india_daily=india_daily.groupby(['Date'])[['Confirmed','Recovered','Deaths','Active']].sum().reset_index()

india_daily['Daily Confirmed']=india_daily['Confirmed'].sub(india_daily['Confirmed'].shift())
india_daily['Daily Confirmed'].iloc[0]=india_daily['Confirmed'].iloc[0]
india_daily['Daily Confirmed'] = india_daily['Daily Confirmed'].astype(int)

india_daily['Daily Recovered']=india_daily['Recovered'].sub(india_daily['Recovered'].shift())
india_daily['Daily Recovered'].iloc[0]=india_daily['Recovered'].iloc[0]
india_daily['Daily Recovered'] = india_daily['Daily Recovered'].astype(int)

india_daily['Daily Deceased']=india_daily['Deaths'].sub(india_daily['Deaths'].shift())
india_daily['Daily Deceased'].iloc[0]=india_daily['Deaths'].iloc[0]
india_daily['Daily Deceased'] = india_daily['Daily Deceased'].astype(int)

india_daily['Daily Active']=india_daily['Active'].sub(india_daily['Active'].shift())
india_daily['Daily Active'].iloc[0]=india_daily['Active'].iloc[0]
india_daily['Daily Active'] = india_daily['Daily Active'].astype(int)


In [ ]:
fig = px.bar(india_daily, y='Daily Confirmed', x='Date',hover_data =['Daily Confirmed'], color='Daily Confirmed', height=500)
fig.update_layout(
    title='Daily Confirmed Cases in India Datewise')
fig.show()

fig = px.bar(india_daily, y='Daily Recovered', x='Date',hover_data =['Daily Recovered'], color='Daily Recovered', height=500)
fig.update_layout(
    title='Daily Recovered Cases in India Datewise')
fig.show()

fig = px.bar(india_daily, y='Daily Deceased', x='Date',hover_data =['Daily Deceased'], color='Daily Deceased', height=500)
fig.update_layout(
    title='Daily Deceased Cases in India Datewise')
fig.show()

fig = px.bar(india_daily, y='Daily Active', x='Date',hover_data =['Daily Active'], color='Daily Active', height=500)
fig.update_layout(
    title='Daily Active Cases in India Datewise')
fig.show()

In [ ]:
#Bar Chart
fig = go.Figure()
fig.add_trace(go.Bar(x=india_daily['Date'],y=india_daily['Daily Confirmed'],
                        marker_color='blue',name='Daily Confirmed'))
fig.add_trace(go.Bar(x=india_daily['Date'],y=india_daily['Daily Deceased'],
                        marker_color='red',name='Daily Deceased'))
fig.add_trace(go.Bar(x=india_daily['Date'],y=india_daily['Daily Recovered'],
                        marker_color='green',name='Daily Recovered'))
fig.add_trace(go.Bar(x=india_daily['Date'],y=india_daily['Daily Active'],
                        marker_color='orange',name='Daily Active'))

fig.update_layout(title_text = 'Covid19 INDIA Daily Trend Bar chart')
fig.show()

### Age

In [ ]:
fig = make_subplots(rows=2,cols=1,specs=[[{"type":"pie"}],[{"type":"bar"}]],
                   subplot_titles=('Pie chart',"Bar Chart"))

#Add traces
fig.add_trace(go.Pie(labels=age_group['AgeGroup'],values=age_group['TotalCases'],hole=0.3),row=1,col=1)

fig.add_trace(go.Bar(x=age_group['AgeGroup'],y=age_group['TotalCases']),row=2,col=1)

fig.update_xaxes(title_text="Age Group", row=2, col=1)
fig.update_yaxes(title_text="Total Cases", row=1, col=1)

# Update title and height
fig.update_layout(title_text="Age Group Visualizations", height=700)

### Testing data

#### Statewise testing Labs

In [ ]:
temp=ICMRstatetesting.groupby(['State'])[['TotalSamples','Negative','Positive']].max().reset_index()
#Total Samples
fig=px.bar(temp.sort_values(by='TotalSamples'),x='TotalSamples',y='State',
          text='TotalSamples',orientation='h',
          height=500,title='States with highest number of samples tested',color='TotalSamples')
fig.update_traces(textposition='outside')
fig.show()
#Negative
fig=px.bar(temp.sort_values(by='Negative'),x='Negative',y='State',
          text='Negative',orientation='h',
          height=500,title='States with highest number of negative sample results',color='Negative')
fig.update_traces(textposition='outside')
fig.show()
#Recovered cases
fig=px.bar(temp.sort_values(by='Positive'),x='Positive',y='State',
          text='Positive',orientation='h',
          height=500,title='States with highest number of positive sample results',color='Positive')
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
#Total no of samples tested in each state

temp=ICMRstatetesting.groupby(['State'])[['TotalSamples','Negative','Positive']].max().reset_index()

temp=temp.sort_values(by='TotalSamples')
fig = go.Figure(data=[
go.Bar(name='TotalSamples', y=temp['State'], x=temp['TotalSamples'], 
       orientation='h',marker_color='blue'),
go.Bar(name='Negative', y=temp['State'], x=temp['Negative'], 
       orientation='h',marker_color='green'),
go.Bar(name='Positive', y=temp['State'], x=temp['Positive'], 
       orientation='h',marker_color='red')])
fig.update_layout(barmode='stack',width=1000, height=800)

fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.update_layout(title_text='Total samples tested vs Positive and Negative',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
ICMRtestinglabs.head()

**How many labs are there in each state which are approved by ICMR**

In [ ]:
temp = ICMRtestinglabs.groupby(['state'])[['lab']].count().reset_index()

fig=px.bar(temp.sort_values(by='lab'),x='lab',y='state',
          color='lab',orientation='h',height=800,text='lab',
            title = 'No of labs in each state')
fig.update_traces(textposition='outside')
fig.show()

#### Population

In [ ]:
#Let's look at the most populated states and how they are affected.
#Is population really a factor in the spread of the virus

temp = india.groupby(['State'])[['Population','Rural population','Urban population','Confirmed','Recovered',
                                'Deaths','Active']].max().reset_index()

temp.head()

In [ ]:
#Subplots
#Population vs Confirmed
fig_c = px.scatter(temp,x='Population',y='Confirmed',color='Confirmed',size='Population',log_x =True,
                   hover_data=['State'])
#Population vs Recovered
fig_r = px.scatter(temp,x='Population',y='Recovered',color='Recovered',size='Population',log_x =True,
                  hover_data=['State'])
#Population vs Deaths
fig_d = px.scatter(temp,x='Population',y='Deaths',color='Deaths',size='Population',log_x =True,
                  hover_data=['State'])
#Population vs Active
fig_a = px.scatter(temp,x='Population',y='Active',color='Active',size='Population',log_x =True,
                  hover_data=['State'])

fig = make_subplots(rows=2,cols=2,horizontal_spacing=0.14, vertical_spacing=0.08,
                   subplot_titles=('Pop vs Confirmed','Pop vs Recovered','Pop vs Deaths','Pop vs Active'))

fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_r['data'][0],row=1,col=2)
fig.add_trace(fig_d['data'][0],row=2,col=1)
fig.add_trace(fig_a['data'][0],row=2,col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Confirmed",row=1,col=1)
fig.update_yaxes(title_text="Recovered",row=1,col=2)
fig.update_yaxes(title_text="Deaths",row=2,col=1)
fig.update_yaxes(title_text="Active",row=2,col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Population",row=1,col=1)
fig.update_xaxes(title_text="Population",row=1,col=2)
fig.update_xaxes(title_text="Population",row=2,col=1)
fig.update_xaxes(title_text="Population",row=2,col=2)

fig.update_layout(height=1200)

#### Rural vs Urban Population

In [ ]:
#Create a bar plot of states with highest urban/rural population and visualize the number of cases
temp.head()

In [ ]:
temp = temp.sort_values(by=['Urban population'])

fig = go.Figure(data=[go.Bar(name='Confirmed',x=temp['Confirmed'],y=temp['State'],orientation='h'),
                     go.Bar(name='Deaths',x=temp['Deaths'],y=temp['State'],orientation='h')])
fig.update_layout(barmode='group',title_text='States with highest urban pop vs Cases',height=900)
fig.show()

temp = temp.sort_values(by=['Rural population'])

fig = go.Figure(data=[go.Bar(name='Confirmed',x=temp['Confirmed'],y=temp['State'],orientation='h'),
                     go.Bar(name='Deaths',x=temp['Deaths'],y=temp['State'],orientation='h')])
fig.update_layout(barmode='group',title_text='States with highest rural pop vs Cases',height=900)
fig.show()

#### Area

In [ ]:
temp = india.groupby(['State'],as_index=False)[['Area','Confirmed','Deaths','Recovered','Active']].max().reset_index(drop=True)

#Subplots
#Area vs Confirmed
fig_c = px.scatter(temp,x='Area',y='Confirmed',color='Confirmed',size='Area',log_x =True,
                   hover_data=['State','Area','Confirmed'])
#Area vs Recovered
fig_r = px.scatter(temp,x='Area',y='Recovered',color='Recovered',size='Area',log_x =True,
                  hover_data=['State','Area','Recovered'])
#Area vs Deaths
fig_d = px.scatter(temp,x='Area',y='Deaths',color='Deaths',size='Area',log_x =True,
                  hover_data=['State','Area','Deaths'])
#Area vs Active
fig_a = px.scatter(temp,x='Area',y='Active',color='Active',size='Area',log_x =True,
                  hover_data=['State','Area','Active'])

fig = make_subplots(rows=2,cols=2,horizontal_spacing=0.14, vertical_spacing=0.08,
                   subplot_titles=('Area vs Confirmed','Area vs Recovered','Area vs Deaths','Area vs Active'))

fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_r['data'][0],row=1,col=2)
fig.add_trace(fig_d['data'][0],row=2,col=1)
fig.add_trace(fig_a['data'][0],row=2,col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Confirmed",row=1,col=1)
fig.update_yaxes(title_text="Recovered",row=1,col=2)
fig.update_yaxes(title_text="Deaths",row=2,col=1)
fig.update_yaxes(title_text="Active",row=2,col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Area",row=1,col=1)
fig.update_xaxes(title_text="Area",row=1,col=2)
fig.update_xaxes(title_text="Area",row=2,col=1)
fig.update_xaxes(title_text="Area",row=2,col=2)

fig.update_layout(height=1200)


#### Density

In [ ]:
temp = india.groupby(['State'],as_index=False)[['Density','Confirmed','Deaths','Recovered','Active']].max().reset_index(drop=True)

#Subplots
#Density vs Confirmed
fig_c = px.scatter(temp,x='Density',y='Confirmed',color='Confirmed',size='Density',log_x =True,
                   hover_data=['State','Density','Confirmed'])
#Density vs Recovered
fig_r = px.scatter(temp,x='Density',y='Recovered',color='Recovered',size='Density',log_x =True,
                  hover_data=['State','Density','Recovered'])
#Density vs Deaths
fig_d = px.scatter(temp,x='Density',y='Deaths',color='Deaths',size='Density',log_x =True,
                  hover_data=['State','Density','Deaths'])
#Density vs Active
fig_a = px.scatter(temp,x='Density',y='Active',color='Active',size='Density',log_x =True,
                  hover_data=['State','Density','Active'])

fig = make_subplots(rows=2,cols=2,horizontal_spacing=0.14, vertical_spacing=0.08,
                   subplot_titles=('Density vs Confirmed','Density vs Recovered','Density vs Deaths','Density vs Active'))

fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_r['data'][0],row=1,col=2)
fig.add_trace(fig_d['data'][0],row=2,col=1)
fig.add_trace(fig_a['data'][0],row=2,col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Confirmed",row=1,col=1)
fig.update_yaxes(title_text="Recovered",row=1,col=2)
fig.update_yaxes(title_text="Deaths",row=2,col=1)
fig.update_yaxes(title_text="Active",row=2,col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Density",row=1,col=1)
fig.update_xaxes(title_text="Density",row=1,col=2)
fig.update_xaxes(title_text="Density",row=2,col=1)
fig.update_xaxes(title_text="Density",row=2,col=2)

fig.update_layout(height=1200)

#### Hospitals

#### Unfortunately we don't have Maharashtra state's data

In [ ]:
temp = india.groupby(['State'])[['AdmissionCapacity','HospitalBeds','Confirmed',
                         'Recovered','Deaths']].max().reset_index()
#Which state has highest number of hospital beds to treat patients
fig = px.bar(temp.sort_values(by='HospitalBeds'),x='HospitalBeds',
            y='State',color='HospitalBeds',orientation='h',height=900,
             hover_data=['State','Confirmed','Recovered','Deaths'],
            title = 'HospitalBeds')
fig.show()
#Admission Capacity
fig = px.bar(temp.sort_values(by='AdmissionCapacity'),x='AdmissionCapacity',
            y='State',color='AdmissionCapacity',orientation='h',height=900,
             hover_data=['State','Confirmed','Recovered','Deaths'],
            title = 'AdmissionCapacity')
fig.show()

### India :- Red , Orange, Green Zones

The government on 1st May announced a two-week extension of the nationwide lockdown to stop the spread of the coronavirus. 

Announced on March 25, the lockdown will now go on till at least May 18. However, it announced several relaxations especially for districts which have been categorised as Green or Orange zones, indicating no or low number of COVID-19 cases.

On Friday, the government said the number of coronavirus or COVID-19 "Red Zones", across India have dropped by around 23 per cent in 15 days, from 170 on April 15 to 130 on April 30. However, all four major metro cities - Delhi, Kolkata, Chennai and Mumbai - have been classified as Red Zones.

In the same period, the number of "green zones", or districts reporting no new coronavirus cases in the last 21 days, also dropped from 356 to 319, while the number of "orange zones" increased - from 207 to 284, according to the Union Health Ministry.

In [ ]:
URL = 'https://www.ndtv.com/india-news/coronavirus-full-list-of-red-orange-green-districts-in-india-2221473'
page = requests.get(URL)
soup = BeautifulSoup(page.content,"html.parser")

'''
Parsing through HTML to extract the table
'''

id_statewise = soup.find("table",{"id":"stateWise"})
id_statewise

tbody = id_statewise.find("tbody") 

#Fetching each row

table_rows = tbody.find_all("tr")

ls = []
for tr in table_rows:
    data_list = tr.find_all("td")
    row = [data.text for data in data_list]
    ls.append(row)
    
#Creating a data frame
india_zones = pd.DataFrame(data = ls,columns = ["State","Red","Orange","Green","Total"])

for col in india_zones.columns:
    if col != 'State':
        india_zones[col] = pd.to_numeric(india_zones[col])

In [ ]:
india_zones.head()

In [ ]:
#Zones distribution in each state
fig=px.bar(india_zones.sort_values(by='Red'),x='Red',y='State',
          text='Red',orientation='h',color_discrete_sequence=['red'],
          height=800,title='States with number of red zones')
fig.update_traces(textposition='outside')
fig.show()
#Negative
fig=px.bar(india_zones.sort_values(by='Orange'),x='Orange',y='State',
          text='Orange',orientation='h',
          height=800,title='States with number of orange zones',color_discrete_sequence=['orange'])
fig.update_traces(textposition='outside')
fig.show()
#Recovered cases
fig=px.bar(india_zones.sort_values(by='Green'),x='Green',y='State',
          text='Green',orientation='h',
          height=800,title='States with number of green zones',color_discrete_sequence=['green'])
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
#Zones distribution in India

india_zones = india_zones[0:-1]

fig = go.Figure(data=[
go.Bar(name='Red Zone', y=india_zones['State'], x=india_zones['Red'], 
       orientation='h',marker_color='red'),
go.Bar(name='Orange Zone', y=india_zones['State'], x=india_zones['Orange'], 
       orientation='h',marker_color='orange'),
go.Bar(name='Green Zone', y=india_zones['State'], x=india_zones['Green'], 
       orientation='h',marker_color='green')])
fig.update_layout(barmode='stack',width=1000, height=800)

fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.update_layout(title_text='Zones distribution in India',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()

### India district wise zones distribution

In [ ]:
URL = 'https://www.ndtv.com/india-news/coronavirus-full-list-of-red-orange-green-districts-in-india-2221473'
page = requests.get(URL)
soup = BeautifulSoup(page.content,"html.parser")

'''
Parsing through HTML to extract the table
'''

id_districtwise = soup.find("table",{"id":"distWise"})
id_districtwise

tbody = id_districtwise.find("tbody")

#Fetching each row

table_rows = tbody.find_all("tr")

ls=[]

for tr in table_rows:
    data_list = tr.find_all("td")
    row = [data.text for data in data_list]
    ls.append(row)
    
india_dist_zones = pd.DataFrame(data=ls,columns = ["Sr.No","District" , "State" , "Classification"])

In [ ]:
india_dist_zones.head()

In [ ]:
#India district coordinates
dist_coor = pd.read_csv("/kaggle/input/indian-census-data-with-geospatial-indexing/district wise centroids.csv")

In [ ]:
dist_coor

In [ ]:
india_dist_zones = india_dist_zones[['District','State','Classification']]
dist_coor = dist_coor[['District','Latitude','Longitude']]
zones_merged = pd.merge(india_dist_zones,dist_coor,on=['District'],how='inner')

### Visualizations of Zones

In [ ]:
#px.set_mapbox_access_token(secret_value_1)
fig = px.scatter_mapbox(zones_merged,
                        lat="Latitude",
                        lon="Longitude",
                        color='Classification',
                        color_discrete_sequence=['red','orange','green'],
                        mapbox_style='open-street-map',
                        hover_name='District',
                        center={"lat": 20.5937, "lon": 78.9629},
                        zoom=3.5,
                        hover_data=['State','District'],
                        title= 'Zones Distribution in India')
fig.update_geos(fitbounds="locations", visible=True)
fig.update_geos(projection_type="orthographic")
fig.update_layout(template='plotly_dark',margin=dict(l=20,r=20,t=40,b=20))
fig.show()

##### Finally ,we are done

#### I'll keep updating this notebook with more data and more visualizations.

#### Let's fight COVID-19 together and

##### Stay Home , Stay Safe India